# Web Scraping von Inhalten der Wikipedia-Seite über Bundeskanzler:innen

Alle benötigten Bibliotheken importieren:

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL aufrufen und HTML-Quellcode der Webseite laden:

In [2]:
url = "https://de.wikipedia.org/wiki/Bundeskanzler_(Deutschland)"
r = requests.get(url)
html = r.text

Ein paar Checks:

In [3]:
print(r.url) # Welche URL wurde angefragt?
print(r.status_code) # Welcher HTTP-Status-Code wurde vom Web-Server zurückgemeldet?
print(r.reason) # Was bedeutet der Status-Code?

https://de.wikipedia.org/wiki/Bundeskanzler_(Deutschland)
200
OK


Der HTML-Quellcode wird an BeautifulSoup (Bibliothek zum Parsen von HTML-Dokumenten) übergeben.

In [4]:
bs = BeautifulSoup(html)

Die abgerufenen Daten sind HTML-Quellcode. Die Struktur der Daten ist wie folgt:

In [5]:
print(bs.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="de">
 <head>
  <meta charset="utf-8"/>
  <title>
   Bundeskanzler (Deutschland) – Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","Januar","Februar","März","April","Mai","Juni","Juli","August","September","Oktober","November","Dezember"],"wgRequestId":"c2657557-8fdc-4c8d-a1ad-2e898234c5bb","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Bundeskanzler_(Deutschland)","wgTitle":"Bundeskanzler (Deutschland)","wgCurRevisionId":228001461,"wgRevisionId":228001461,"wgArticleId":618,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Seite mit täglicher Aktualisierung","Wikipedia:Artikel mit Video","Wikipedia:Gesproc

Mit Hilfe von BeautifulSoup ist es nun möglich, die gesuchten Informationen aus dem HTML-Code zu extrahieren. Spezifische Elemente des Quellcodes werden anhand von [CSS-Selektoren](https://www.w3schools.com/cssref/css_selectors.asp) ausgewählt und in Variablen gespeichert. CSS-Selektoren ermöglichen die präzise Auswahl von Elementen im HTML-Code.

Beispielsweise kann man alle Anchor-Elemente mit Hyperlinks (`<a href="..."> </a>`) aus dem HTML-Quellcode auslesen:

In [7]:
link_elements = bs.select('a[href]')
link_texts = [l.text for l in link_elements] # alternativ: link_texts = [l.get_text() for l in linkelements]
links = [l['href'] for l in link_elements] # alternativ: links = [l.get_attribute_list("href")[0] for l in linkelements]

In [8]:
for l in links:
    print(l)

#Vorlage_Gesprochene_Version
#mw-head
#searchInput
/wiki/Datei:DEgov-BK-Logo.svg
/wiki/Bundeswappen_Deutschlands#Bundesrepublik_Deutschland_seit_1949
/wiki/Datei:Flag_of_the_Chancellor_of_Germany.svg
/wiki/Standarte
/wiki/Datei:2021-09-12_Politik,_TV-Triell_Bundestagswahl_2021_1DX_3801_by_Stepro_(cropped).jpg
/wiki/Olaf_Scholz
/wiki/Anrede
#cite_note-:1-1
/wiki/Amtssitz
/wiki/Bundeskanzleramt_(Berlin)
/wiki/Berlin
/wiki/Palais_Schaumburg
/wiki/Bonn
/wiki/Beh%C3%B6rde
/wiki/Bundeskanzleramt_(Deutschland)
/wiki/Amtszeit
#Ende_der_Amtszeit
/wiki/Vizekanzler_(Deutschland)
/wiki/Abstimmungen_%C3%BCber_den_deutschen_Bundeskanzler
/wiki/Wahl_zum_21._Deutschen_Bundestag
/wiki/Bundestag
/wiki/Grundgesetz_f%C3%BCr_die_Bundesrepublik_Deutschland
/wiki/Konrad_Adenauer
/wiki/Website
https://www.bundeskanzler.de/
#cite_note-2
/wiki/Regierungschef
/wiki/Deutschland
/wiki/Bundesregierung_(Deutschland)
/wiki/Richtlinienkompetenz#Bundeskanzler
/wiki/Koalition_(Politik)
/wiki/Bundespr%C3%A4sident_(Deutsc

Auslesen der Informationen aus der Tabelle über alle Bundeskanzler:innen seit dem 2. Weltkrieg:

In [9]:
names = bs.select('.wikitable > tbody > tr > td:nth-child(4)')
parties = bs.select('.wikitable > tbody > tr > td:nth-child(5)')
inaug = bs.select('.wikitable > tbody > tr > td:nth-child(6)')
end = bs.select('.wikitable > tbody > tr > td:nth-child(7)')
length = bs.select('.wikitable > tbody > tr > td:nth-child(8)')

Text aus den ausgewählten Elementen extrahieren, aufräumen und in einem Dictionary speichern:

In [10]:
chancellors = {
    'name': [n.get_text().replace('\n', ' ').replace('Anm. 3', '').strip() for n in names],
    'party': [p.get_text().replace('\n', ' ').replace('Anm. 2', '').strip() for p in parties],
    'inaug': [i.get_text().replace('\n', ' ').strip() for i in inaug],
    'end': [e.get_text().replace('\n', ' ').strip() for e in end],
    'length': [l.get_text().replace('\n', ' ').strip() for l in length]
}

Datensatz erstellen:

In [11]:
df = pd.DataFrame.from_dict(chancellors)
df.head(10)

,name,party,inaug,end,length
0,Konrad Adenauer (1876–1967),CDU,15. September 1949,16. Oktober 1963,"14 Jahre, 1 Monat, 1 Tag (5144 Tage)"
1,Ludwig Erhard (1897–1977),CDU,16. Oktober 1963,1. Dezember 1966,"3 Jahre, 1 Monat, 15 Tage (1142 Tage)"
2,Kurt Georg Kiesinger (1904–1988),CDU,1. Dezember 1966,21. Oktober 1969,"2 Jahre, 10 Monate, 20 Tage (1055 Tage)"
3,Willy Brandt (1913–1992),SPD,21. Oktober 1969,7. Mai 1974,"4 Jahre, 6 Monate, 16 Tage (1659 Tage)"
4,Walter Scheel(geschäftsführend) (1919–2016),FDP,7. Mai 1974,16. Mai 1974,9 Tage
5,Helmut Schmidt (1918–2015),SPD,16. Mai 1974,1. Oktober 1982,"8 Jahre, 4 Monate, 15 Tage (3060 Tage)"
6,Helmut Kohl (1930–2017),CDU,1. Oktober 1982,27. Oktober 1998,"16 Jahre, 26 Tage (5870 Tage)"
7,Gerhard Schröder (* 1944),SPD,27. Oktober 1998,22. November 2005,"7 Jahre, 26 Tage (2583 Tage)"
8,Angela Merkel (* 1954),CDU,22. November 2005,8. Dezember 2021,"16 Jahre, 16 Tage (5860 Tage)"
9,Olaf Scholz (* 1958),SPD,8. Dezember 2021,im Amt,1 Jahr und 6 Tage (371 Tage)


Neue Variable erstellen, die die Amtszeit in Tagen als ganzzahligen Wert enthält:

In [12]:
df['days'] = df['length'].str.extract(r'\(?(\d+) Tage\)?$').astype(int)

Datensatz als CSV-Datei abspeichern:

In [13]:
df.to_csv('chancellors_ger.csv')  

Datensatz nach Amtszeit in Tagen absteigend sortieren:

In [14]:
df.sort_values(by = ['days'], ascending = False, inplace = True)
df.head(10)

,name,party,inaug,end,length,days
6,Helmut Kohl (1930–2017),CDU,1. Oktober 1982,27. Oktober 1998,"16 Jahre, 26 Tage (5870 Tage)",5870
8,Angela Merkel (* 1954),CDU,22. November 2005,8. Dezember 2021,"16 Jahre, 16 Tage (5860 Tage)",5860
0,Konrad Adenauer (1876–1967),CDU,15. September 1949,16. Oktober 1963,"14 Jahre, 1 Monat, 1 Tag (5144 Tage)",5144
5,Helmut Schmidt (1918–2015),SPD,16. Mai 1974,1. Oktober 1982,"8 Jahre, 4 Monate, 15 Tage (3060 Tage)",3060
7,Gerhard Schröder (* 1944),SPD,27. Oktober 1998,22. November 2005,"7 Jahre, 26 Tage (2583 Tage)",2583
3,Willy Brandt (1913–1992),SPD,21. Oktober 1969,7. Mai 1974,"4 Jahre, 6 Monate, 16 Tage (1659 Tage)",1659
1,Ludwig Erhard (1897–1977),CDU,16. Oktober 1963,1. Dezember 1966,"3 Jahre, 1 Monat, 15 Tage (1142 Tage)",1142
2,Kurt Georg Kiesinger (1904–1988),CDU,1. Dezember 1966,21. Oktober 1969,"2 Jahre, 10 Monate, 20 Tage (1055 Tage)",1055
9,Olaf Scholz (* 1958),SPD,8. Dezember 2021,im Amt,1 Jahr und 6 Tage (371 Tage),371
4,Walter Scheel(geschäftsführend) (1919–2016),FDP,7. Mai 1974,16. Mai 1974,9 Tage,9
